<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/Model_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Install sitk library to read dicom files
!pip install SimpleITK  
import pandas as pd
import os
import SimpleITK as sitk
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import numpy as np
train = pd.read_csv(r"/content/drive/My Drive/Project_Boneage/train.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import cv2

In [4]:
def read_image(path):
  reader = sitk.ImageSeriesReader()

  img_names = cv2.imread(path)
  reader.SetFileNames(img_names)

  image = reader.Execute()

  image = sitk.GetArrayFromImage(image)
  #some images in the dataset had a different size, this would have to be 
  #pre-processed using registration or simply cropping as done below
  image = image[0:20,0:512,0:512]
  
  return(image)

In [5]:
class CombineDataset(Dataset):

    def __init__(self, frame, id_col, label_name, path_imgs, transform=None):
        """
        Args:
            frame (pd.DataFrame): Frame with the tabular data.
            id_col (string): Name of the column that connects image to tabular data
            label_name (string): Name of the column with the label to be predicted
            path_imgs (string): path to the folder where the images are.
            transform (callable, optional): Optional transform to be applied
                on a sample, you need to implement a transform to use this.
        """
        self.frame = frame
        self.id_col = id_col
        self.label_name = label_name
        self.path_imgs = path_imgs
        #self.transform = transform

    def __len__(self):
        return (self.frame.shape[0])

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        #complete image path and read
        img_name = self.frame[self.id_col].iloc[idx]
        path = os.path.join(self.path_imgs,img_name)
        image = read_image(path)
        image = torch.from_numpy(image.astype(np.float32))

        #get the other features to be used as training data
        feats = [feat for feat in self.frame.columns if feat not in [self.label_name,self.id_col]]
        feats  = np.array(self.frame[feats].iloc[idx])
        feats = torch.from_numpy(feats.astype(np.float32))
       
        
        #get label
        label = np.array(self.frame[self.label_name].iloc[idx])
        label = torch.from_numpy(label.astype(np.float32))
        #label = torch.as_tensor(np.array(label).astype('float'))

        return image, feats, label

In [6]:
train['id'] = train['id'].apply(lambda x: str(x)+'.png')
train['boneage'] = train['boneage'].apply(lambda x: str(x))

In [7]:
path_imgs = r"/content/drive/My Drive/Project_Boneage/boneage_training_dataset"
train_set = CombineDataset(train,'id','boneage','male',path_imgs)

loader_trainer = DataLoader(
    train_set,
    batch_size = 2,
    shuffle = True,
    num_workers = 0,
    drop_last=True
)

In [8]:
for data in loader_trainer:
    x,z, y = data
    
    print('Image Loaded Size = ', x.shape)
    print('Tabular data = ', z)
    print('Label = ', y)

ValueError: ignored

# Model

# Training